In [3]:
import pickle
import numpy as np

In [4]:
ratings = pickle.load(open('items_10000_matrix.pickle', 'rb'), encoding='iso-8859-1')

In [5]:
type(ratings)

scipy.sparse.csr.csr_matrix

In [6]:
ratings = ratings.toarray()

In [7]:
ratings.shape

(71869, 10000)

*ratings* -  np.array состоящий из нулей и единиц, где каждая строка соответсвует пользователю, а каждая колонка - книге.  
1 - на пересечении строки i колонки j значит, что пользователь i прочитал книгу j. 0 - пользователь i не читал книгу j.

# Part 1

### Task 1. Найти самую популярную книгу.

In [15]:
book_readers_count = ratings.sum(axis=0)
most_popular_book = book_readers_count.argmax()
print('most popular book = {} (numeration from 0)'.format(most_popular_book))

most popular book = 20 (numeration from 0)


### Task 2. Найти пользователя со второй по величине библиотекой.

In [24]:
readers_library = ratings.sum(axis=1)
for i in readers_library:
    print(i, ' ', end='')
print()    
print(readers_library[69648])
second_biggest_library = (readers_library.argsort()[-2], readers_library.argsort()[-1])
print('user_id with the second biggest library = {}'.format(second_biggest_library))

2  3  1  1  0  3  1  3  2  2  4  4  0  2  6  6  3  4  1  0  2  3  6  0  4  4  4  1  4  1  2  0  1  2  3  2  1  0  1  5  8  4  3  1  3  1  2  3  1  1  1  4  2  4  2  0  2  0  5  2  0  1  3  1  2  3  2  1  3  2  2  1  5  1  2  3  3  2  2  4  2  6  2  3  1  2  3  2  3  0  0  1  2  0  2  0  2  1  3  2  2  3  2  3  1  3  2  2  1  2  5  3  5  0  3  2  2  0  2  4  4  3  1  1  1  5  0  4  2  1  3  1  2  3  2  0  4  3  2  1  4  3  0  1  3  4  5  1  0  2  1  3  1  1  1  5  4  5  2  2  1  3  4  3  1  4  3  1  4  2  1  3  5  4  1  3  0  3  4  2  3  2  2  6  3  2  3  2  2  3  4  1  4  2  2  1  1  3  2  3  2  1  4  4  3  1  1  2  0  2  1  2  2  4  2  1  2  3  1  3  0  4  3  3  0  2  0  5  3  0  2  5  1  2  4  1  3  5  2  2  2  0  0  2  3  1  1  3  1  1  2  2  3  1  2  2  2  1  4  1  2  1  8  2  1  6  0  2  0  1  0  2  2  3  5  3  4  1  1  4  3  2  1  1  1  5  3  1  4  1  2  2  0  3  5  1  1  1  5  4  4  2  2  3  0  4  5  2  5  4  2  2  4  2  1  3  1  4  3  1  2  4  1  0  4  1  4  5  4  1  3  2  0  2

1  6  2  4  3  2  2  5  2  2  2  1  2  2  1  2  2  1  0  5  2  4  4  0  4  2  1  1  3  5  0  3  1  0  2  4  2  8  1  0  1  1  1  1  0  1  1  3  2  2  5  1  1  2  1  2  5  2  2  4  1  2  1  3  2  3  2  7  2  3  3  4  2  2  3  2  4  3  2  0  2  4  2  0  2  2  2  3  2  2  1  1  0  2  2  0  3  3  1  3  1  2  2  2  6  9  2  2  1  2  4  1  4  2  6  1  1  4  2  0  2  2  2  4  1  2  4  1  2  5  3  3  1  1  5  0  4  5  1  3  4  1  4  1  2  2  2  2  3  1  3  4  1  5  2  4  5  1  6  2  1  1  4  1  3  2  1  1  4  5  1  1  2  3  3  2  0  5  2  4  2  1  2  4  2  1  1  1  2  3  4  6  4  2  1  3  0  1  3  2  2  4  2  3  1  1  4  3  1  0  1  4  0  1  3  1  5  5  2  1  4  1  2  3  0  1  3  1  4  1  3  2  5  1  4  3  3  1  3  1  1  0  5  2  4  3  0  3  3  2  7  3  1  0  2  0  2  2  1  1  2  1  0  2  2  2  1  7  1  2  2  3  1  2  4  0  0  0  0  2  2  0  2  0  3  2  5  4  3  4  4  2  5  2  3  2  4  1  3  2  2  2  1  1  0  2  3  2  2  2  1  3  3  0  4  1  1  1  3  1  1  1  4  0  1  3  4  3  1  1  1  1  3  1

4  3  2  2  3  1  4  2  7  1  0  1  6  2  3  0  2  4  0  2  4  2  3  3  5  2  2  4  1  0  4  2  2  2  2  6  3  1  1  1  1  4  5  2  2  2  3  2  3  0  1  3  4  2  2  2  1  3  1  5  1  3  0  4  3  3  2  2  1  0  2  2  2  0  4  1  2  0  0  4  0  0  3  4  3  2  1  1  4  4  3  0  1  3  4  1  4  5  1  3  2  2  3  2  1  3  3  4  2  1  2  0  2  4  2  1  2  3  3  1  3  4  3  3  3  3  3  0  1  3  4  1  1  2  0  1  4  6  1  2  2  2  4  3  1  1  2  2  4  3  7  4  3  3  3  1  3  1  3  5  1  4  2  3  1  0  2  3  2  4  2  2  4  1  3  3  1  0  3  2  5  2  3  2  4  1  1  1  2  2  2  3  2  1  4  1  2  2  1  3  0  0  6  4  4  3  4  2  2  1  1  2  3  2  2  2  4  1  2  2  4  2  3  4  3  2  2  2  3  2  4  2  9  4  1  2  0  3  5  1  3  2  3  3  3  2  2  3  0  4  0  5  2  2  4  2  1  2  2  0  3  3  3  1  2  4  1  7  2  1  3  5  6  4  3  1  4  0  3  1  4  3  1  3  3  4  3  5  3  0  3  1  3  2  4  1  2  5  4  2  3  2  3  3  3  1  2  2  1  3  0  2  2  2  3  1  5  2  2  1  1  2  2  3  2  3  1  1  4  1  1  1  2  2

1  2  2  2  4  5  1  1  2  1  3  3  2  1  5  1  1  5  1  0  2  1  2  2  2  4  0  2  2  1  6  0  5  3  0  5  3  1  5  2  3  2  4  1  2  4  3  4  4  2  2  1  4  2  0  1  0  5  3  2  0  3  4  5  3  2  0  0  2  1  1  0  2  2  1  2  1  3  5  0  2  2  6  1  2  2  4  6  2  0  1  1  2  5  3  1  0  2  1  2  2  0  3  1  1  3  0  2  4  1  4  2  4  1  3  3  1  0  3  1  2  4  2  1  3  3  1  3  5  4  1  5  0  2  3  1  2  1  1  0  2  2  3  6  3  2  1  3  1  1  2  2  0  2  2  4  2  4  5  1  0  1  4  4  1  3  1  3  3  2  2  3  3  2  2  0  2  2  2  4  0  4  2  3  6  2  3  2  1  2  3  1  3  3  2  2  5  1  1  4  1  4  4  2  2  6  3  3  3  3  3  5  3  0  3  1  1  0  4  1  0  2  2  5  4  2  3  0  0  4  0  2  2  1  4  0  2  0  5  3  3  1  3  3  2  1  2  5  2  1  0  3  4  4  1  0  1  2  2  3  5  1  2  1  2  2  1  0  3  2  4  2  3  1  3  1  0  4  4  1  2  1  2  2  1  7  2  4  0  4  1  2  1  1  1  2  2  0  2  2  0  4  3  2  6  1  4  3  1  1  3  5  1  5  7  4  5  2  2  1  3  5  4  1  8  2  5  5  2  3  1  1  1  1

1  1  2  4  1  5  3  3  5  0  2  2  1  1  0  2  2  1  3  3  2  0  1  1  3  4  1  1  3  2  1  2  4  2  4  0  1  3  1  0  1  3  2  4  2  2  3  3  3  3  2  3  2  2  2  5  5  3  1  4  1  2  4  1  2  3  4  2  3  1  0  1  0  2  2  1  2  1  4  2  7  0  1  1  2  2  3  1  4  0  0  0  1  4  1  7  3  3  5  0  6  4  0  2  1  0  1  3  1  0  2  3  2  0  1  0  1  0  3  2  4  0  6  4  2  3  2  1  1  2  1  2  2  3  0  2  3  5  2  1  1  3  3  1  3  2  3  1  5  3  1  1  3  3  1  4  8  1  3  4  4  2  2  4  2  0  3  3  4  1  3  2  3  3  1  1  3  2  2  2  2  3  4  1  1  2  4  0  3  2  3  0  5  2  5  3  1  0  3  5  1  0  3  1  4  1  2  4  2  2  2  2  3  1  3  3  2  2  3  1  1  2  1  0  1  4  2  1  1  5  3  1  2  4  5  1  4  1  5  2  0  3  2  3  2  4  1  2  4  4  5  3  7  3  1  3  9  0  3  3  2  2  1  1  2  2  2  4  2  2  0  3  1  2  0  2  2  4  1  2  3  1  4  2  4  0  0  4  2  3  2  2  2  2  2  2  1  2  2  1  4  2  3  1  3  3  0  2  2  2  1  4  3  2  1  3  2  1  2  2  2  3  1  4  3  6  3  3  4  1  3  2  1  1

...

3  1  0  3  1  3  7  2  4  4  3  5  1  5  1  2  2  3  1  0  1  1  2  3  2  0  4  5  0  4  1  2  2  1  3  3  0  6  5  2  1  2  1  1  2  5  2  3  3  2  1  4  1  5  4  2  2  1  2  2  4  5  4  5  3  3  1  2  1  3  4  5  1  1  5  1  0  3  5  4  3  2  2  3  1  3  3  4  1  1  1  3  1  1  1  2  1  1  2  2  2  1  0  2  4  1  2  3  1  2  0  0  0  9  2  1  2  1  2  2  5  1  1  1  4  2  5  2  1  4  2  1  0  2  2  2  2  2  2  0  3  4  1  0  1  5  3  1  3  4  7  2  2  0  3  1  4  2  2  6  0  3  5  6  1  1  2  4  2  3  1  3  3  0  1  5  5  5  3  1  3  4  2  3  7  5  1  4  2  3  2  6  0  4  5  1  2  4  6  3  1  2  6  2  2  5  4  3  6  1  1  2  1  0  5  3  3  4  2  4  2  3  1  3  3  5  1  1  1  4  0  0  2  1  4  4  3  2  3  2  3  4  5  2  3  1  2  1  1  3  3  3  2  3  1  1  2  3  3  3  2  3  4  1  2  5  2  4  5  3  0  4  3  5  2  0  4  3  2  1  0  4  1  2  3  2  3  0  4  1  1  4  2  3  1  3  2  2  2  2  2  0  2  5  3  0  2  0  2  5  5  5  0  5  1  3  3  2  8  2  2  1  1  0  3  0  2  1  5  1  1  3  4  4

 5  1  2  2  0  2  2  3  3  3  2  4  1  0  2  3  2  4  2  2  3  3  3  2  4  0  3  5  0  0  1  1  3  4  3  3  4  0  3  3  2  1  0  2  0  3  1  2  3  3  2  3  2  2  3  2  5  2  2  4  1  3  2  4  5  0  3  0  3  1  2  1  5  1  2  4  0  3  4  2  2  0  1  3  1  2  4  2  3  1  4  4  1  2  2  3  0  3  0  2  1  1  1  1  1  5  0  6  3  1  0  5  1  4  1  2  1  1  1  1  1  4  3  4  0  5  1  3  3  1  3  0  0  3  1  5  4  2  4  1  0  3  2  2  3  6  2  3  3  1  3  1  0  3  3  4  2  2  2  3  4  4  2  1  1  1  2  2  2  2  4  1  0  0  7  3  2  0  2  2  3  0  2  2  0  2  1  6  1  2  2  1  5  1  3  0  3  3  5  4  4  4  1  2  4  2  4  2  4  1  4  2  2  0  4  0  5  3  5  0  2  2  6  2  2  2  3  3  1  1  2  2  1  2  7  3  2  2  2  2  1  2  2  0  2  3  4  2  3  2  1  7  1  1  2  2  4  4  2  1  4  2  0  3  1  1  0  2  2  4  1  5  2  1  5  2  5  3  3  3  0  0  1  2  6  3  0  2  4  3  3  1  2  2  3  1  4  4  1  1  3  5  2  1  5  1  1  1  2  1  1  3  1  3  2  4  2  3  1  0  3  2  2  5  5  4  3  1  2  0  1  4  1  

  0  4  3  1  1  3  2  1  5  1  2  3  3  4  1  3  0  2  2  5  3  4  0  2  0  2  3  1  2  1  3  5  1  0  4  2  0  4  3  4  4  2  4  3  3  1  2  3  1  4  4  1  4  2  4  2  2  1  3  2  3  4  2  4  1  2  1  2  1  5  1  1  4  1  1  1  3  2  4  0  2  1  3  4  3  2  4  4  2  3  2  3  3  0  3  3  1  1  4  2  3  3  1  2  5  3  2  5  3  0  4  2  1  1  2  3  4  2  4  1  3  4  3  4  4  2  1  1  2  3  0  3  2  2  1  0  3  1  1  2  0  4  2  3  2  2  4  3  0  1  4  2  3  2  1  2  0  3  4  5  3  0  1  2  1  2  1  5  2  1  6  1  5  4  0  4  2  3  0  2  0  0  3  2  0  2  3  0  4  4  2  1  2  2  4  4  1  6  5  3  2  1  3  1  2  4  4  5  4  4  0  0  3  3  0  0  1  0  4  1  4  3  1  5  1  1  5  1  7  4  3  3  3  3  2  3  2  3  3  4  4  4  3  5  3  4  5  3  1  0  3  4  0  2  0  2  3  3  0  1  2  2  4  4  0  2  2  3  3  4  3  5  3  3  1  2  5  1  1  2  2  5  2  5  5  4  2  2  2  4  2  2  6  1  1  2  0  1  4  2  2  2  0  2  1  2  4  1  3  3  1  2  1  2  2  0  2  1  2  2  2  4  2  1  1  4  1  2  1  0  7  1  3 

 2  0  2  4  1  2  0  3  5  3  5  0  6  1  2  0  2  2  3  3  5  2  6  2  3  1  4  2  2  2  2  2  3  0  1  3  1  1  3  0  1  3  1  2  6  0  2  4  2  2  1  5  0  2  2  2  5  2  1  1  2  1  3  4  1  3  2  1  5  0  1  3  2  3  1  2  1  1  1  1  2  1  3  4  1  3  1  2  5  0  1  5  3  0  1  2  4  4  3  1  0  0  1  1  4  2  3  0  1  4  0  3  1  0  1  1  0  1  2  0  4  3  4  3  2  3  0  6  4  3  3  2  1  1  2  6  3  0  2  2  3  3  5  2  2  2  2  2  1  0  5  2  5  2  5  2  1  1  1  3  3  2  2  3  2  2  2  1  4  5  3  1  0  3  3  2  1  1  0  2  2  3  3  2  5  1  2  3  1  6  2  2  4  1  2  0  2  2  5  0  2  2  3  2  1  3  3  1  3  1  3  5  1  3  1  4  3  2  4  2  1  0  5  1  3  4  3  2  3  1  2  6  2  0  0  2  1  1  4  3  5  1  0  4  3  2  0  2  3  1  1  1  5  1  2  0  1  1  3  1  7  3  3  3  4  0  1  3  2  1  4  1  3  1  2  2  3  1  4  4  2  0  3  4  3  4  1  0  2  0  3  2  1  2  3  0  2  3  1  1  4  1  1  6  2  1  3  5  3  2  2  1  4  0  4  2  3  2  2  0  4  1  2  5  1  0  0  1  3  1  4  2  4  

  2  2  2  1  1  1  0  6  2  0  2  2  2  3  4  1  5  4  4  0  2  0  2  3  1  4  2  2  0  1  2  0  2  1  1  2  1  0  1  3  4  2  1  5  3  2  1  1  0  2  3  3  3  1  0  0  1  0  4  0  1  4  2  6  1  2  2  2  3  1  2  0  0  2  4  2  3  1  5  2  4  4  4  4  1  4  1  1  3  2  0  2  3  1  5  1  1  1  5  4  3  3  1  4  2  1  0  2  0  0  4  2  4  3  2  2  1  1  1  4  1  1  4  4  1  3  4  1  0  6  2  3  0  4  4  2  1  3  4  4  5  5  3  1  5  3  6  2  4  2  2  1  2  3  0  3  6  1  2  1  1  3  4  3  4  1  2  3  1  1  0  3  2  0  2  3  2  3  2  2  2  4  3  2  3  1  1  4  1  3  5  2  4  2  5  0  2  0  2  4  2  2  2  1  2  2  2  5  3  2  1  1  2  5  2  0  2  0  5  1  2  1  2  2  1  2  3  0  3  3  1  2  7  4  6  2  2  1  4  1  2  2  3  3  0  3  1  3  2  3  2  0  3  3  1  4  4  3  1  2  2  0  2  0  1  1  1  2  4  2  1  2  2  2  3  1  4  2  2  0  3  1  4  2  2  2  3  5  1  2  1  1  2  4  1  3  3  2  5  4  0  4  2  3  1  2  3  3  2  2  1  1  1  0  2  2  3  1  3  2  1  1  1  2  2  1  3  5  3  3  1  2  1 

### Task 3. Найти среднее и медиану размера библиотеки.

In [21]:
mean = readers_library.mean()
print('mean = {}'.format(mean))

median = np.median(readers_library)
print('median = {}'.format(median))

mean = 2.266693567463023
median = 2.0


### Task 4. Найти медиану размера библиоткеи у пользователей, которые прочитали книгу номер с id 100.

In [39]:
readers_100_library = list((x.sum() for x in ratings if x[100] == 1))
median_number = np.median(readers_100_library)
print('Median number of books of users who read the book with id 100 = {}'.format(median_number))

Median number of books of users who read the book with id 100 = 3.0


# Part 2

### Task 1. Похожие книги.
1. Найди 20 наиболее похожих книг для книги с id = 100. В качестве меры похожести используй cosine similarity. Используй наиболее быстрый вариант подсчета.
  
$$cosine\ similarity = \frac{A\cdot B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n} A_iB_i}{\sqrt{\sum_{i=1}^{n}A_i^2}\sqrt{\sum_{i=1}^{n}B_i^2}}$$  
  
В качестве ответа верни список кортежей, где каждый кортеж представляет собой пару (book_id, similarity).

In [ ]:
similarity = []

for i in ratings.shape()[1]:
    

# Part 3

### Task 1. Матрица похожих книг (топ-20).

Создать матрицу похожих книг (матрица 10000 x 10000) использую метрику *cosine similarity*. В каждоый строке оставить только топ 20 самых похожих. Лучше написать функцию которая на вход получает матрицу рейтингов на выходе матрица похожих книг. Эту фунцкцию можно будет использовать в следующем задании.

In [ ]:
def compute_sim_matrix(ratings):
    pass

### Task 2. Оценить рекомендательную систему.

Оценить качество рекомендательной системы, основанной на матрице похожих книг, использую метрики **precision** и **recall**. Рекомендации должны получаться путем умножения матрицы похожих книг на вектор пользователя.
  
**precison** - кол-во книг которые попали пользователю в рекомендации и он их действительно добавил делить на кол-во рекомендованных книг.
  
**recall** - кол-во книг которые попали пользователю в рекомендации и он их действительно добавил делить на кол-во книг добавленных пользователем. 
  
Метрики считать на 20 самых лучших рекомендациях для каждого пользователя.